In [13]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderUnavailable

# Initialize Nominatim API with a specific user-agent
geolocator = Nominatim(user_agent="geoloc_crexplore")

# Predefined set of cities with their coordinates
cities = {
    "Berlin": geolocator.geocode("Berlin, Germany"),
    "Munich": geolocator.geocode("Munich, Germany"),
    "Hamburg": geolocator.geocode("Hamburg, Germany"),
    "Cologne": geolocator.geocode("Cologne, Germany"),
    "Frankfurt": geolocator.geocode("Frankfurt, Germany"),
    "Stuttgart": geolocator.geocode("Stuttgart, Germany"),
    "Leipzig": geolocator.geocode("Leipzig, Germany"),
    "Mannheim": geolocator.geocode("Mannheim, Germany"),
}

# Assuming cities have been geocoded properly, we extract their coordinates
city_coords = {city: (city_data.latitude, city_data.longitude) for city, city_data in cities.items()}
#import locations csv
import pandas as pd
df = pd.read_csv('/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/dataframes/locations_startup.csv',sep=';')
locations = df['startup_location'].tolist()
#take the first 10 from locations

print(locations)
# Example list of city names

# Function to find the nearest city within a radius or 'other'
# Function to find the nearest city within a radius or 'other'
def find_nearest_city(location_name, city_coords, radius=80):
    try:
        # Geocode the location to get its coordinates
        location = geolocator.geocode(location_name)
        if not location:
            return "unknown"  # Could not geocode the location

        # Check if the location is in Germany using the country_code attribute
        location_country_code = geolocator.reverse((location.latitude, location.longitude), language='en').raw['address']['country_code']
        if location_country_code.lower() != 'de':
            return "international"

        # Calculate the distance to each city and return the nearest city within the radius
        for city, coords in city_coords.items():
            if geodesic((location.latitude, location.longitude), coords).km <= radius:
                return city
        return "other"
    except GeocoderTimedOut:
        return "timeout"  # Geocoding service timed out
    except GeocoderUnavailable:
        return "unavailable"  # Geocoding service is unavailable
    except Exception as e:
        return "error"  # Other geocoding errors


# #parallelize the function
# from joblib import Parallel, delayed
# import multiprocessing
# num_cores = multiprocessing.cpu_count()
# results = Parallel(n_jobs=num_cores)(delayed(find_nearest_city)(location, city_coords) for location in locations)
# print(results)


# Map each location to a city, 'international', 'unknown', or 'other'
mapped_locations_startup = {location: find_nearest_city(location, city_coords) for location in locations}

print(mapped_locations_startup)

['Berlin', 'München', 'Hamburg', 'Köln', 'Düsseldorf', 'Frankfurt am Main', 'Stuttgart', 'Karlsruhe', 'Dresden', 'Aachen', 'Hannover', 'Leipzig', 'Mannheim', 'Nürnberg', 'Potsdam', 'Münster', 'Darmstadt', 'Bremen', 'Heidelberg', 'Bonn', 'Dortmund', 'Freiburg im Breisgau', 'Mainz', 'Essen', 'Bochum', 'Kiel', 'Augsburg', 'Grünwald', 'Saarbrücken', 'Bielefeld', 'Braunschweig', 'Jena', 'Wiesbaden', 'Regensburg', 'Paderborn', 'Würzburg', 'Ulm', 'Chemnitz', 'Wuppertal', 'Magdeburg', 'Göttingen', 'Erlangen', 'Fürth', 'Konstanz', 'Osnabrück', 'Rostock', 'Tübingen', 'Oldenburg', 'Krefeld', 'Bad Homburg v. d. Höhe', 'Planegg', 'Gräfelfing', 'Ludwigsburg', 'Halle (Saale)', 'Mönchengladbach', 'Lübeck', 'Ingolstadt', 'Kaiserslautern', 'Siegen', 'Garching b.München', 'Gilching', 'Offenburg', 'Mülheim an der Ruhr', 'Bergisch Gladbach', 'Koblenz', 'Duisburg', 'Solingen', 'Ismaning', 'Gießen', 'Taufkirchen', 'Unterföhring', 'Monheim am Rhein', 'Trier', 'Heilbronn', 'Leverkusen', 'Ratingen', 'Langenfeld

In [15]:
#save mapped locatons to csv
df['mapped_location'] = df['startup_location'].map(mapped_locations_startup)
df.to_csv('/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/dataframes/locations_startup_mapped.csv',sep=';',index=False)

In [11]:
#read from csv
df = pd.read_csv('/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/dataframes/locations_mapped.csv',sep=';')
#count per mapped_location
df['mapped_locations'].value_counts()
#multiply 


international    1210
other            1199
unknown           627
Munich            356
Frankfurt         337
Stuttgart         323
Cologne           295
Hamburg           164
Berlin            110
Leipzig            68
Mannheim           59
unavailable         4
error               1
Name: mapped_locations, dtype: int64